In [3]:
import requests
import warnings
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# convert to config file/table
CV = 'Integration_ORD-CostCenterHierarchy_Normalised'
urls = [f'https://api.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}', f'https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}']

iteration = 1 
for link in urls:
    
    global df, fields, fieldtypes, rjson
    df = None
    fields = None
    fieldtypes = None
    rjson = None
    if iteration == 1:
        with open(r'C:\Users\gmoye001\config\apiconnect.json') as f:
            headers = json.load(f)
    elif iteration == 2:
        with open(r'C:\Users\gmoye001\config\apiconnectb.json') as f:
            headers = json.load(f)
    
    print(link)
    try:         
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', FutureWarning)
            r = requests.get(link, headers=headers)
            rjson = r.json()
            #print(rjson)
            keylist = ('URI','Categories','RelatedTerms')
            ekey = ('ErrorMessage')
            
            #if rjson == "{'ErrorMessage': 'CV does not exist in CVMaster List'}":
                #print('CV Missing from CVMaster List')
                #logging.info(f'\n{rjson} - CV is temporarily unavailable\n')
            #else:
            if 'ErrorMessage' in rjson:
                if rjson['ErrorMessage'] == 'CV does not exist in CVMaster List':
                    print('CV Currently Missing from CVMaster List')
            else:
                for key in keylist:
                    rjson = [{k: v for k, v in d.items() if k != key} for d in rjson]
                    
                    df = pd.DataFrame(rjson)
                    display(df)
                    datelist = ('CreatedDate','ModifiedDate','EffectiveDate', 'HierarchyNodeEffectiveDate', 'HierarchyNodeExpirationDate','RelModifiedDate','ExpiryDate','Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime')
                    date_format = "%Y%m%d%H%M%S"
                    
                    for date in enumerate(datelist): 
                        if date[1] in df:
                            
                            if df[date[1]].items() != 'None':
                                if df[date[1]].str.contains('Z').items():
                                    df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
                            if df[date[1]].items() != 'None':   
                                if df[date[1]].str.contains('-').items():
                                    df[date[1]] = df[date[1]].str.replace("\W+", "")
                                    
                            df[date[1]] = df[date[1]].mask(df[date[1]].str.len() > 14, df[date[1]].str[:-3])
                            df[date[1]] = pd.to_datetime(df[date[1]], format=date_format, errors = 'coerce')
                            
                        else:
                            continue
            
            fields = df.columns.tolist()
            #create_table = 
            fields = "','".join(map(str, fields))
            fields = "'"+fields+"'"
            fieldtypes = df.dtypes
            fieldtypes = pd.DataFrame(data=fieldtypes).reset_index(drop = False, inplace = False)
            fieldtypes = fieldtypes.rename(columns={'index':'Fields', 0:'dtypes'})
            createtable = ''
            for row, cell in fieldtypes.iterrows():
                createfield = str(cell[0]) + ' ' + str(cell[1]).replace('object','nvarchar(255) NULL,').replace('datetime64[ns]','datetime NULL,')
                print(createfield)
            print('\n',fields,'\n')
            display(fieldtypes)
            print('\n')
            
    except (NameError, AttributeError) as ne:
        print('\n', ne, '\n', "CV probably doesn't exist in prod or stage :(")
        pass
    
    iteration += 1
    #display(df)
    print('\n')

https://api.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/Integration_ORD-CostCenterHierarchy_Normalised
CV Currently Missing from CVMaster List

 'NoneType' object has no attribute 'columns' 
 CV probably doesn't exist in prod or stage :(


https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/Integration_ORD-CostCenterHierarchy_Normalised


,NetworkNodeID,NetworkNodeName,NodeId,NodeCode,NodeName,NodeUniversalCode,HierarchyNodeID,HierarchyNodeName,HierarchyNodeRecordStatus,HierarchyNodeEffectiveDate,HierarchyNodeExpirationDate,HierarchyId,HierarchyName,ParentNetworkNodeID,ParentNetworkNodeName,ParentNodeId,ParentNodeCode,ParentNodeName,ParentNodeUniversalCode,ActiveStatus,CreatedDate,ModifiedDate
0,12716280,Japan TLS,7833,W2_HCM_CC_JPN_TLS_Common_CC01,TLS Common - Overseas,XRW2_HCM_CC_JPN_TLS_Common_CC01,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2578,W2_HCM_CC_JPN_L1_TLS - TLS_Common_01,TLS Common - Overseas,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_01,deactivated,20180807015038.520,20200103030025.193
1,12716280,Japan TLS,7836,W2_HCM_CC_JPN_TLS_Common_CC04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_TLS_Common_CC04,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,deactivated,20180807015038.520,20200103030025.193
2,12716280,Japan TLS,7837,W2_HCM_CC_JPN_TLS_Common_CC05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_TLS_Common_CC05,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2582,W2_HCM_CC_JPN_L1_TLS - TLS_Common_05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_05,deactivated,20180807015038.520,20200103030025.193
3,12716280,Japan TLS,2588,W2_HCM_CC_JPN_L1_TLS - TLS_Common_11,TLS Common - CAS,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_11,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
4,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63174,1974663,PwC Liechtenstein,22056392,Test 5,Test 5,LITest 5,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63175,1974663,PwC Liechtenstein,22056391,Test 4,Test 4,LITest 4,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63176,1974663,PwC Liechtenstein,22056386,C3,C3,LIC3,18609577,Local LoS,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056387,C4,C4,LIC4,active,20230217143121.633,20230217143121.633
63177,1974666,PwC Germany,22056404,A2,A2,DEA2,18609577,Local LoS,active,None,None,18386249,Standard,1974666,PwC Germany,22056405,A3,A3,DEA3,active,20230220152139.587,20230220152139.587


,NetworkNodeID,NetworkNodeName,NodeId,NodeCode,NodeName,NodeUniversalCode,HierarchyNodeID,HierarchyNodeName,HierarchyNodeRecordStatus,HierarchyNodeEffectiveDate,HierarchyNodeExpirationDate,HierarchyId,HierarchyName,ParentNetworkNodeID,ParentNetworkNodeName,ParentNodeId,ParentNodeCode,ParentNodeName,ParentNodeUniversalCode,ActiveStatus,CreatedDate,ModifiedDate
0,12716280,Japan TLS,7833,W2_HCM_CC_JPN_TLS_Common_CC01,TLS Common - Overseas,XRW2_HCM_CC_JPN_TLS_Common_CC01,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2578,W2_HCM_CC_JPN_L1_TLS - TLS_Common_01,TLS Common - Overseas,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_01,deactivated,20180807015038.520,20200103030025.193
1,12716280,Japan TLS,7836,W2_HCM_CC_JPN_TLS_Common_CC04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_TLS_Common_CC04,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,deactivated,20180807015038.520,20200103030025.193
2,12716280,Japan TLS,7837,W2_HCM_CC_JPN_TLS_Common_CC05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_TLS_Common_CC05,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2582,W2_HCM_CC_JPN_L1_TLS - TLS_Common_05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_05,deactivated,20180807015038.520,20200103030025.193
3,12716280,Japan TLS,2588,W2_HCM_CC_JPN_L1_TLS - TLS_Common_11,TLS Common - CAS,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_11,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
4,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63174,1974663,PwC Liechtenstein,22056392,Test 5,Test 5,LITest 5,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63175,1974663,PwC Liechtenstein,22056391,Test 4,Test 4,LITest 4,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63176,1974663,PwC Liechtenstein,22056386,C3,C3,LIC3,18609577,Local LoS,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056387,C4,C4,LIC4,active,20230217143121.633,20230217143121.633
63177,1974666,PwC Germany,22056404,A2,A2,DEA2,18609577,Local LoS,active,None,None,18386249,Standard,1974666,PwC Germany,22056405,A3,A3,DEA3,active,20230220152139.587,20230220152139.587


,NetworkNodeID,NetworkNodeName,NodeId,NodeCode,NodeName,NodeUniversalCode,HierarchyNodeID,HierarchyNodeName,HierarchyNodeRecordStatus,HierarchyNodeEffectiveDate,HierarchyNodeExpirationDate,HierarchyId,HierarchyName,ParentNetworkNodeID,ParentNetworkNodeName,ParentNodeId,ParentNodeCode,ParentNodeName,ParentNodeUniversalCode,ActiveStatus,CreatedDate,ModifiedDate
0,12716280,Japan TLS,7833,W2_HCM_CC_JPN_TLS_Common_CC01,TLS Common - Overseas,XRW2_HCM_CC_JPN_TLS_Common_CC01,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2578,W2_HCM_CC_JPN_L1_TLS - TLS_Common_01,TLS Common - Overseas,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_01,deactivated,20180807015038.520,20200103030025.193
1,12716280,Japan TLS,7836,W2_HCM_CC_JPN_TLS_Common_CC04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_TLS_Common_CC04,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,deactivated,20180807015038.520,20200103030025.193
2,12716280,Japan TLS,7837,W2_HCM_CC_JPN_TLS_Common_CC05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_TLS_Common_CC05,18609576,Local cost centre,active,None,None,18386249,Standard,12716280,Japan TLS,2582,W2_HCM_CC_JPN_L1_TLS - TLS_Common_05,TLS Common - Tokyo Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_05,deactivated,20180807015038.520,20200103030025.193
3,12716280,Japan TLS,2588,W2_HCM_CC_JPN_L1_TLS - TLS_Common_11,TLS Common - CAS,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_11,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
4,12716280,Japan TLS,2581,W2_HCM_CC_JPN_L1_TLS - TLS_Common_04,TLS Common - Kasumiga-seki Office,XRW2_HCM_CC_JPN_L1_TLS - TLS_Common_04,18609577,Local LoS,deactivated,None,None,18386249,Standard,12716280,Japan TLS,2595,W2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,Tax and Legal Services,XRW2_HCM_CC_JPN_L2_Tax and Legal Services_TLS,deactivated,20180807015038.257,20200103031026.380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63174,1974663,PwC Liechtenstein,22056392,Test 5,Test 5,LITest 5,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63175,1974663,PwC Liechtenstein,22056391,Test 4,Test 4,LITest 4,18609576,Local cost centre,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056380,B1,B1,LIB1,active,20230220093058.557,20230220093058.557
63176,1974663,PwC Liechtenstein,22056386,C3,C3,LIC3,18609577,Local LoS,active,None,None,18386249,Standard,1974663,PwC Liechtenstein,22056387,C4,C4,LIC4,active,20230217143121.633,20230217143121.633
63177,1974666,PwC Germany,22056404,A2,A2,DEA2,18609577,Local LoS,active,None,None,18386249,Standard,1974666,PwC Germany,22056405,A3,A3,DEA3,active,20230220152139.587,20230220152139.587


NetworkNodeID nvarchar(255) NULL,
NetworkNodeName nvarchar(255) NULL,
NodeId nvarchar(255) NULL,
NodeCode nvarchar(255) NULL,
NodeName nvarchar(255) NULL,
NodeUniversalCode nvarchar(255) NULL,
HierarchyNodeID nvarchar(255) NULL,
HierarchyNodeName nvarchar(255) NULL,
HierarchyNodeRecordStatus nvarchar(255) NULL,
HierarchyNodeEffectiveDate datetime NULL,
HierarchyNodeExpirationDate datetime NULL,
HierarchyId nvarchar(255) NULL,
HierarchyName nvarchar(255) NULL,
ParentNetworkNodeID nvarchar(255) NULL,
ParentNetworkNodeName nvarchar(255) NULL,
ParentNodeId nvarchar(255) NULL,
ParentNodeCode nvarchar(255) NULL,
ParentNodeName nvarchar(255) NULL,
ParentNodeUniversalCode nvarchar(255) NULL,
ActiveStatus nvarchar(255) NULL,
CreatedDate datetime NULL,
ModifiedDate datetime NULL,

 'NetworkNodeID','NetworkNodeName','NodeId','NodeCode','NodeName','NodeUniversalCode','HierarchyNodeID','HierarchyNodeName','HierarchyNodeRecordStatus','HierarchyNodeEffectiveDate','HierarchyNodeExpirationDate','Hierar

,Fields,dtypes
0,NetworkNodeID,object
1,NetworkNodeName,object
2,NodeId,object
3,NodeCode,object
4,NodeName,object
5,NodeUniversalCode,object
6,HierarchyNodeID,object
7,HierarchyNodeName,object
8,HierarchyNodeRecordStatus,object
9,HierarchyNodeEffectiveDate,datetime64[ns]


In [6]:
fieldtypes = fieldtypes.rename(columns={'index':'Fields', 0:'dtypes'})
display(fieldtypes)

,Fields,dtypes
0,NetworkNodeID,object
1,NetworkNodeName,object
2,NodeId,object
3,NodeCode,object
4,NodeName,object
5,NodeUniversalCode,object
6,HierarchyNodeID,object
7,HierarchyNodeName,object
8,HierarchyNodeRecordStatus,object
9,HierarchyNodeEffectiveDate,datetime64[ns]


NetworkNodeID nvarchar(255),
NetworkNodeName nvarchar(255),
NodeId nvarchar(255),
NodeCode nvarchar(255),
NodeName nvarchar(255),
NodeUniversalCode nvarchar(255),
HierarchyNodeID nvarchar(255),
HierarchyNodeName nvarchar(255),
HierarchyNodeRecordStatus nvarchar(255),
HierarchyNodeEffectiveDate datetime,
HierarchyNodeExpirationDate datetime,
HierarchyId nvarchar(255),
HierarchyName nvarchar(255),
ParentNetworkNodeID nvarchar(255),
ParentNetworkNodeName nvarchar(255),
ParentNodeId nvarchar(255),
ParentNodeCode nvarchar(255),
ParentNodeName nvarchar(255),
ParentNodeUniversalCode nvarchar(255),
ActiveStatus nvarchar(255),
CreatedDate datetime,
ModifiedDate datetime,


In [7]:
import pyodbc
#Currently determining which datatypes need to be used
#stackoverflow.com/questions/20229441/implications-of-nvarchar-50-vs-nvarchar-max
#stackoverflow.com/questions/35366/what-are-the-main-performance-differences-between-varchar-and-nvarchar-sql-serve
#stackoverflow.com/questions/612430/when-must-we-use-nvarchar-nchar-instead-of-varchar-char-in-sql-server
def create_tables():
    
    """create tables in the SQL Server Database"""
    commands = (
    """ 
    CREATE TABLE [GCV_PRD].[NS-ManagementLevel-en-NS-JobLevel-en_NS-ResourceRole-en_Normalised] (
    Id nvarchar(20),
    Name nvarchar(50),
    ParentId nvarchar(20),
    ParentVocabName nvarchar(50),
    RelModifiedDate datetime,
    ResourceRoleName nvarchar(50),
    Definition nvarchar(255),
    EffectiveDate datetime,
    ExpiryDate datetime,
    LastEditedReason nvarchar(20),
    Comments nvarchar(255),
    Systemofrecord nvarchar(50)
    )
    """,
    """
    CREATE TABLE [GCV_STG].[NS-ManagementLevel-en-NS-JobLevel-en_NS-ResourceRole-en_Normalised] (
    Id nvarchar(20),
    Name nvarchar(50),
    ParentId nvarchar(20),
    ParentVocabName nvarchar(50),
    RelModifiedDate datetime,
    ResourceRoleName nvarchar(50),
    Definition nvarchar(255),
    EffectiveDate datetime,
    ExpiryDate datetime,
    LastEditedReason nvarchar(20),
    Comments nvarchar(255),
    Systemofrecord nvarchar(50)
    )
    """,)
    conn = None
    try:
        server = 'GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com'
        database = 'GCV_API'
        conn = pyodbc.connect('Trusted_Connection=yes',DRIVER='{SQL Server}',SERVER=server,DATABASE=database)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the SQL Server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, pyodbc.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
if __name__ == '__main__':
    create_tables()

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(df)

,Id,Name,ParentId,ParentVocabName,RelModifiedDate,ResourceRoleName,Definition,EffectiveDate,ExpiryDate,LastEditedReason,Comments,Systemofrecord
0,18277094,Administrative,12682942,NS-JobLevel-en,2020-03-19 10:56:28,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
1,18277094,Administrative,12669416,NS-ManagementLevel-en,2020-03-19 10:56:26,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
2,18277094,Administrative,12669419,NS-JobLevel-en,2020-03-19 10:56:29,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
3,18277094,Administrative,12669420,NS-JobLevel-en,2020-03-19 10:56:29,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
4,18277094,Administrative,12669421,NS-JobLevel-en,2020-03-19 10:56:29,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
5,18277094,Administrative,12669422,NS-JobLevel-en,2020-03-19 10:56:29,Administrative,Administrative staff roles include Administrat...,2020-03-18,NaT,New term added,None,Synaptica
6,18277095,Associate,12682943,NS-JobLevel-en,2020-03-19 10:56:29,Associate,Associate staff roles,2020-03-18,NaT,New term added,None,Synaptica
7,18277095,Associate,1840265,NS-ManagementLevel-en,2020-03-19 10:56:29,Associate,Associate staff roles,2020-03-18,NaT,New term added,None,Synaptica
8,18277096,Associate 2,12669423,NS-JobLevel-en,2020-03-19 10:56:29,Associate 2,Associate 2 staff roles,2020-03-18,NaT,New term added,None,Synaptica
9,18277097,Director,12682948,NS-JobLevel-en,2020-03-19 10:56:29,Director,Director staff roles,2020-03-18,NaT,New term added,None,Synaptica


In [33]:
datelist = ('CreatedDate','ModifiedDate','EffectiveDate', 'HierarchyNodeEffectiveDate', 'HierarchyNodeExpirationDate', 'RelModifiedDate', 'ExpiryDate', 'Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime')
date_format = "%Y%m%d%H%M%S"

for date in enumerate(datelist): 
    if date[1] in df:
        
        if df[date[1]].iteritems() != 'None':
            if df[date[1]].str.contains('Z').items():
                df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
        if df[date[1]].iteritems() != 'None':   
            if df[date[1]].str.contains('-').items():
                df[date[1]] = df[date[1]].str.replace("\W+", "")
                
        df[date[1]] = df[date[1]].mask(df[date[1]].str.len() > 14, df[date[1]].str[:-3])
        df[date[1]] = pd.to_datetime(df[date[1]], format=date_format, errors = 'coerce')
        
    else:
        continue
        
global recnum
recnum = len(df.index)

pd.set_option('display.max_columns', None)

#df = df.iloc[:1000]
display(df)#[['HierarchyNodeEffectiveDate', 'HierarchyNodeExpirationDate']])


,NetworkNodeID,NetworkNodeName,NodeId,NodeCode,NodeName,NodeUniversalCode,HierarchyNodeID,HierarchyNodeName,HierarchyNodeRecordStatus,HierarchyNodeEffectiveDate,HierarchyNodeExpirationDate,HierarchyId,HierarchyName,ParentNetworkNodeID,ParentNetworkNodeName,ParentNodeId,ParentNodeCode,ParentNodeName,ParentNodeUniversalCode,ActiveStatus,CreatedDate,ModifiedDate
0,1974698,PwC Denmark,22551,D10F20,Deals - UDGÅET,DKD10F20,18609576,Local cost centre,active,NaT,NaT,18386249,Standard,1974698,PwC Denmark,20001095,Deals,Deals,DKDeals,active,2018-12-28 14:50:28,2018-12-28 14:50:28
1,1974698,PwC Denmark,22552,D10F21,TS,DKD10F21,18609576,Local cost centre,active,NaT,NaT,18386249,Standard,1974698,PwC Denmark,20001095,Deals,Deals,DKDeals,active,2018-12-28 14:50:28,2018-12-28 14:50:28
2,1974629,PwC Bangladesh,20001218,62010,CEO Office,BD62010,18609577,Local LoS,deactivated,NaT,NaT,18386249,Standard,1974629,PwC Bangladesh,20001179,62000,CEO Office,BD62000,deactivated,2019-01-11 18:50:40,2020-01-11 11:50:24
3,1974629,PwC Bangladesh,20001303,21110,Global Mobility Services,BD21110,18609577,Local LoS,deactivated,NaT,NaT,18386249,Standard,1974629,PwC Bangladesh,20001194,21100,Global Mobility Services,BD21100,deactivated,2019-01-11 18:50:40,2020-01-11 11:50:24
4,1974629,PwC Bangladesh,20001304,21120,Support,BD21120,18609577,Local LoS,deactivated,NaT,NaT,18386249,Standard,1974629,PwC Bangladesh,20001194,21100,Global Mobility Services,BD21100,deactivated,2019-01-11 18:50:40,2020-01-11 11:50:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91087,1974584,PwC Italy,20051449,8500140100,COOSTAFF SAP,IT8500140100,18609577,Local LoS,active,2021-12-01,NaT,18386249,Standard,1974584,PwC Italy,20051448,8500140,COO STAFF,IT8500140,active,2021-12-20 13:00:34,2021-12-20 13:00:34
91088,1974711,PwC United Kingdom,20051534,BU4M,Global Mobility,UKBU4M,18609577,Local LoS,active,NaT,NaT,18386249,Standard,1974711,PwC United Kingdom,20051550,RGB9,UK Galaxy,UKRGB9,active,2021-12-20 14:30:32,2021-12-20 14:30:32
91089,1974696,PwC Ukraine,30213,DOK240,Tax - GCS (DOK240),UADOK240,18609576,Local cost centre,active,NaT,NaT,18386249,Standard,1974558,PwC Central and Eastern Europe (CEE),20006587,TGZTM1L2,Tax - Tax Reporting and Strategy,QZTGZTM1L2,active,2019-05-20 17:20:39,2020-10-12 07:42:41
91090,1974599,PwC Argentina,20013394,ARO671,XB-ASS-Olivos-AAC-Assurance - DAT_ESG US A,ARARO671,18609576,Local cost centre,active,NaT,NaT,18636004,Access control,1974599,PwC Argentina,20065432,XB671,XB-ASS-AAC-Assurance - DAT_ESG US A,ARXB671,active,2022-04-25 19:10:42,2022-08-08 22:50:42
